In [1]:
# Obviously
import pandas as pd
import numpy as np

# Plotting set-up
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
# Command to reset plot styles to default: mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('seaborn-poster')
mpl.rcParams['font.family'] = 'serif'
% matplotlib inline

# Other packages we might need
#! pip install quandl
import quandl

# Options
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
data_folder = 'data/'

In [2]:
import tools
data_folder = '../../data/'
data = tools.read_all_csvs(data_folder)
for f in data.keys():
    cmd = "{} = data['{}']".format(f, f)
    exec(cmd)

In [20]:
data_folder = '../../compdata/'
calendar = pd.read_csv(data_folder + 'calendar.csv.gz')

In [25]:
calendar.shape

(18383955, 5)

In [22]:
calendar.head()

,listing_id,date,available,price,metro_area
0,665257,2016-04-22,f,NaN,asheville
1,665257,2016-04-23,f,NaN,asheville
2,665257,2016-04-24,f,NaN,asheville
3,665257,2016-04-25,f,NaN,asheville
4,665257,2016-04-26,f,NaN,asheville


### Clean calendar

In [23]:
calendar['available'] = calendar['available'].map({'f': False, 't': True})
calendar['price'] = calendar['price'].apply(lambda p: float(str(p).replace('$', '').replace(',', '')))

### Pengji's Listings Cleaning

In [4]:
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype
        
price_columns = ['price', 'weekly_price']
for col in price_columns:
    if not is_numeric_dtype(listings[col]):
        listings[col] = listings[col].str.replace('$', '')
        listings[col] = listings[col].str.replace(',', '')
        listings[col] = listings[col].astype(np.float64)

boolean_columns = ['has_availability','instant_bookable']
for col in boolean_columns:
    for index,value in listings[col].iteritems():
        if listings[col][index]=='t':
            listings.set_value(index, col, 1.0)
        else:
            listings.set_value(index, col, 0.0)

            
amenity=listings['amenities'].apply(lambda x: x.replace('"', '').replace('{', '').replace('}', '').split(','))
amenity_dic={}
for index,value in amenity.iteritems():
    for item in value:
        if item != '':
            if item in amenity_dic.keys():
                amenity_dic[item]=amenity_dic[item]+1
            else:
                amenity_dic[item]=1

sLength=len(listings['id'])
for key in amenity_dic:
    listings[key] = pd.Series(np.zeros(sLength), index=listings.index)

for index,value in amenity.iteritems():
    for item in value:
        listings.set_value(index, item, 1.0)
    
# digitize cancelation policy
sLength=len(listings['id'])
cancelation_policy={}
for index,value in listings['cancellation_policy'].iteritems():
    if value in cancelation_policy.keys():
        cancelation_policy[value]=cancelation_policy[value]+1
    else:
        cancelation_policy[value]=1
        
listings['cancellation_policy_num'] = pd.Series(np.zeros(sLength), index=listings.index)
cancelation_policy_convert={'no_refunds':0.0,'super_strict_60': 1.0,'super_strict_30': 2.0,'strict': 3.0, 'moderate': 4.0,'flexible': 5.0}

for index,value in listings['cancellation_policy'].iteritems():
    listings.set_value(index, 'cancellation_policy_num', cancelation_policy_convert[value])

sLength=len(listings['id'])
bed_type={}
for index,value in listings['bed_type'].iteritems():
    if value in bed_type.keys():
        bed_type[value]=bed_type[value]+1
    else:
        bed_type[value]=1
        
listings['bed_type_num'] = pd.Series(np.zeros(sLength), index=listings.index)
bed_type_convert={'Real Bed': 0, 'Futon': 1, 'Pull-out Sofa': 2, 'Couch': 3, 'Airbed': 4}

for index,value in listings['bed_type'].iteritems():
    listings.set_value(index, 'bed_type_num', bed_type_convert[value])

sLength=len(listings['id'])
room_type={}
for index,value in listings['room_type'].iteritems():
    if value in room_type.keys():
        room_type[value]=room_type[value]+1
    else:
        room_type[value]=1
        
listings['room_type_num'] = pd.Series(np.zeros(sLength), index=listings.index)
room_type_convert={'Shared room': 0, 'Private room': 1, 'Entire home/apt': 2}

for index,value in listings['room_type'].iteritems():
    listings.set_value(index, 'room_type_num', room_type_convert[value])

def format_zipcode(z):
    try:
        fz = int(float(str(z).zfill(5)))
        if len(str(fz)) != 5:
            return False
        else:
            return True
    except ValueError:
        return False
    
listings = listings[listings['zipcode'].apply(lambda z: format_zipcode(z))]


# Format metropolitan areas to names
listings['metropolitan'] = listings['metropolitan'].map({
        'asheville': 'Asheville',
        'austin': 'Austin',
        'nashville': 'Nashville',
        'new_orleans': 'New Orleans',
        'LA': 'Los Angeles'
    })

/usr/lib64/python3.4/site-packages/ipykernel/__main__.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib64/python3.4/site-packages/ipykernel/__main__.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib64/python3.4/site-packages/ipykernel/__main__.py:35: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib64/python3.4/site-packages/ipykernel/__main__.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib64/python3.4/site-packages/ipykernel/__main__.py:64: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib64/python3.4/site-packages/ipykernel/__main__.py:78: FutureWar

### Merge with listings
We want to get the combined calendar data with listings.

In [30]:
data_folder = '../../comp-data/'
estate = pd.read_csv(data_folder + 'real_estate.csv.gz')